In [178]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyvi.ViTokenizer import ViTokenizer
import random
import re
from random import shuffle


In [179]:
#Đọc lại
with open('word_net_vietnamese.json', 'r', encoding='utf-8') as file:
    list_synonyms_vn = json.load(file)



In [180]:
df = pd.read_csv("religion_creed.csv")


In [181]:
STOPWORDS = 'vietnamese-stopwords.txt'

with open(STOPWORDS, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

# Định nghĩa hàm

In [182]:
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line
########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stopwords]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	word = word.replace("_", " ")
	if(word in list_synonyms_vn.keys() and len(list_synonyms_vn[word]) > 0):
		return list_synonyms_vn[word]
	return []

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):

	#obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words
########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)
	

In [183]:
def eda(sentence, alpha_sr=0.5, alpha_ri=0.5, alpha_rs=0.5, p_rd=0.5, num_aug=30):
    # Tokenize câu tiếng Việt
    sentence = ViTokenizer.tokenize(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug / 4) + 1

    # Synonym Replacement (Thay thế từ đồng nghĩa)
    if alpha_sr > 0:
        n_sr = max(1, int(alpha_sr * num_words))
        for _ in range(num_new_per_technique):
            a_words = synonym_replacement(words, n_sr)
            a_words = [a_word.replace("_", " ") for a_word in a_words]
            augmented_sentences.append(' '.join(a_words))

    # Random Insertion (Chèn ngẫu nhiên)
    if alpha_ri > 0:
        n_ri = max(1, int(alpha_ri * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_insertion(words, n_ri)
            a_words = [a_word.replace("_", " ") for a_word in a_words]
            augmented_sentences.append(' '.join(a_words))

    # Random Swap (Hoán đổi ngẫu nhiên)
    if alpha_rs > 0:
        n_rs = max(1, int(alpha_rs * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            a_words = [a_word.replace("_", " ") for a_word in a_words]
            augmented_sentences.append(' '.join(a_words))

    # Random Deletion (Xóa ngẫu nhiên)
    if p_rd > 0:
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, p_rd)
            a_words = [a_word.replace("_", " ") for a_word in a_words]
            augmented_sentences.append(' '.join(a_words))

    # augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    # # shuffle(augmented_sentences)

	# #trim so that we have the desired number of augmented sentences
    # if num_aug >= 1:
    #     augmented_sentences = augmented_sentences[:num_aug]
    # else:
    #     keep_prob = num_aug / len(augmented_sentences)
    #     augmented_sentences = [s for s in augmented_sentences if random.uniform(0,1) < keep_prob]

	#append the original sentence
    augmented_sentences.append(sentence)
    # Trả về các câu đã tăng cường
    return augmented_sentences

In [184]:
df_augumentation = pd.DataFrame(columns=["content", "individual", "groups", "religion/creed", "race/ethnicity", "politics"])
for idx in range(len(df)):
    text_aug = eda(df['content'][idx])
    
    df_merge = pd.DataFrame({
        'content' : text_aug,
        'individual' : [df['individual'][idx]] * len(text_aug),
        'groups' : [df['groups'][idx]] * len(text_aug),
        'religion/creed' : [df['religion/creed'][idx]] * len(text_aug),
        'race/ethnicity' : [df['race/ethnicity'][idx]] * len(text_aug),
        'politics' : [df['politics'][idx]] * len(text_aug)
    })
    df_augumentation = pd.concat([df_augumentation, df_merge], axis = 0)

In [185]:
df_augumentation.to_csv("religion_creed_aug.csv", sep='\t', index=False)